In [ ]:
!pip install awswrangler

In [ ]:
from pathlib import Path  
import pandas as pd  
import awswrangler as wr
from datetime import datetime, timedelta 

In [ ]:
#Note: XO might cause kernel to run out of memory, so will need to run in Athena
s3_output_path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/mmix_weekly_outputs/'
target_database = 'sandbox_measurement_analyst'
target_workgroup = 'cmg-oasis-prod-measurement_analyst-wkgrp'


cur = Path('/root/MARS_ETL_PROCESSES/sql/weekly_grain/etl/')
sql_files = sorted([file for file in cur.iterdir() if str(file).endswith('.sql')])

for sql_file in sql_files:
    table_name = sql_file.stem
    
    with open(sql_file, 'r') as file:
        sql_query = file.read()
    
    df = wr.athena.read_sql_query(
        sql=sql_query,
        database=target_database,
        workgroup=target_workgroup
    )
    
    s3_table_path = f"{s3_output_path}{table_name}/"
    
    wr.s3.to_parquet(
        df=df,
        path=s3_table_path,
        dataset=True,
        mode='overwrite'
    )

    print(f"Data for {table_name} written to {s3_table_path}")
print('done')

In [ ]:
#Sales QTY at Daily Level
def split_sales_and_aggregate_daily(df):
    df['date'] = pd.to_datetime(df['year_month'], format='%Y%m')
    
    df['days_in_month'] = df['date'].dt.days_in_month
    df['daily_sales'] = df['sales_qty'] / df['days_in_month']
    
    df['date_range'] = df.apply(lambda row: pd.date_range(start=row['date'], periods=row['days_in_month'], freq='D'), axis=1)
    df = df.explode('date_range')
    
    daily_sales = df.groupby(['product_brand_name', 'zip', 'date_range'])['daily_sales'].sum().reset_index()
    
    daily_sales['year'] = daily_sales['date_range'].dt.year
    daily_sales['month'] = daily_sales['date_range'].dt.month
    daily_sales['day'] = daily_sales['date_range'].dt.day
    
    return daily_sales[['product_brand_name', 'zip', 'year', 'month', 'day', 'date_range', 'daily_sales']]

query = """select * from sandbox_measurement_analyst.mars_sales_staging_new where year_month between '202201' and '202312'"""
df = wr.athena.read_sql_query(query, database="sandbox_measurement_analyst", workgroup='cmg-oasis-prod-measurement_analyst-wkgrp')
result = split_sales_and_aggregate_daily(df)
wr.s3.to_parquet(
    df=result,
    path='s3://cmg-oasis-prod-sandbox-measurement-analyst/mmix_weekly_outputs/mmix_sales_qty_daily',
    dataset=True,
    database='sandbox_measurement_analyst',
    table='mmix_sales_qty_daily'
)

In [ ]:
#Net Sales at Daily Level 
def split_sales_and_aggregate_daily(df):
    df['date'] = pd.to_datetime(df['year_month'], format='%Y%m')
    
    df['days_in_month'] = df['date'].dt.days_in_month
    df['daily_sales'] = df['net_sales'] / df['days_in_month']
    
    df['date_range'] = df.apply(lambda row: pd.date_range(start=row['date'], periods=row['days_in_month'], freq='D'), axis=1)
    df = df.explode('date_range')
    
    daily_sales = df.groupby(['product_brand_name', 'date_range'])['daily_sales'].sum().reset_index()
    
    daily_sales['year'] = daily_sales['date_range'].dt.year
    daily_sales['month'] = daily_sales['date_range'].dt.month
    daily_sales['day'] = daily_sales['date_range'].dt.day
    
    return daily_sales[['product_brand_name', 'year', 'month', 'day', 'date_range', 'daily_sales']]

query = """select * from sandbox_measurement_analyst.mars_net_sales_staging_new where year_month between '202201' and '202312'"""
df = wr.athena.read_sql_query(query, database="sandbox_measurement_analyst", workgroup='cmg-oasis-prod-measurement_analyst-wkgrp')
result = split_sales_and_aggregate_daily(df)
wr.s3.to_parquet(
    df=result,
    path='s3://cmg-oasis-prod-sandbox-measurement-analyst/mmix_weekly_outputs/mmix_net_sales_daily',
    dataset=True,
    database='sandbox_measurement_analyst',
    table='mmix_net_sales_daily'
)

In [ ]:
#Product ID level sales
def split_sales_and_aggregate_daily(df):
    df['date'] = pd.to_datetime(df['year_month'].astype(str), format='%Y%m')
    
    df['days_in_month'] = df['date'].dt.days_in_month
    
    for col in ['sales_eqv_qty', 'gross_sales_dollars', 'sales_qty_eqv_mce_inc', 'gross_sales_dollars_mce_inc']:
        df[f'daily_{col}'] = df[col] / df['days_in_month']
    
    df['date_range'] = df.apply(lambda row: pd.date_range(start=row['date'], periods=row['days_in_month'], freq='D'), axis=1)
    df = df.explode('date_range')
    
    df['year'] = df['date_range'].dt.year
    df['month'] = df['date_range'].dt.month
    df['week_of_month'] = df['date_range'].dt.day.apply(lambda x: (x - 1) // 7 + 1)
    
    agg_dict = {f'daily_{col}': 'sum' for col in ['sales_eqv_qty', 'gross_sales_dollars', 'sales_qty_eqv_mce_inc', 'gross_sales_dollars_mce_inc']}
    weekly_sales = df.groupby(['year', 'month', 'week_of_month', 'product_brand_name', 'product_id', 'product_description', 'site_zip']).agg(agg_dict).reset_index()
    
    weekly_sales['week'] = weekly_sales.apply(lambda row: 
        pd.to_datetime(f"{row['year']}-{row['month']}-01") + 
        pd.Timedelta(weeks=row['week_of_month']-1, days=-1), axis=1)
    
    for col in ['sales_eqv_qty', 'gross_sales_dollars', 'sales_qty_eqv_mce_inc', 'gross_sales_dollars_mce_inc']:
        weekly_sales = weekly_sales.rename(columns={f'daily_{col}': f'weekly_{col}'})
    
    return weekly_sales[['product_brand_name', 'product_id', 'product_description', 'site_zip', 'year', 'month', 'week', 
                         'weekly_sales_eqv_qty', 'weekly_gross_sales_dollars', 'weekly_sales_qty_eqv_mce_inc', 'weekly_gross_sales_dollars_mce_inc']]

query = """select * from sandbox_measurement_analyst.mars_gne_867_product_sales_staging_new where year_month between '202201' and '202312'"""
df = wr.athena.read_sql_query(query, database="sandbox_measurement_analyst", workgroup='cmg-oasis-prod-measurement_analyst-wkgrp')
result = split_sales_and_aggregate_daily(df)
wr.s3.to_parquet(
    df=result,
    path='s3://cmg-oasis-prod-sandbox-measurement-analyst/mmix_weekly_outputs/mmix_product_sales_daily',
    dataset=True,
    database='sandbox_measurement_analyst',
    table='mmix_product_sales_daily'
)

In [ ]:
s3_output_path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/mmix_weekly_outputs/'
target_database = 'sandbox_measurement_analyst'
target_workgroup = 'cmg-oasis-prod-measurement_analyst-wkgrp'


cur = Path('/root/MARS_ETL_PROCESSES/sql/weekly_grain/sales/')
sql_files = sorted([file for file in cur.iterdir() if str(file).endswith('.sql')])

for sql_file in sql_files:
    table_name = sql_file.stem
    
    with open(sql_file, 'r') as file:
        sql_query = file.read()
    
    df = wr.athena.read_sql_query(
        sql=sql_query,
        database=target_database,
        workgroup=target_workgroup
    )
    
    s3_table_path = f"{s3_output_path}{table_name}/"
    
    wr.s3.to_parquet(
        df=df,
        path=s3_table_path,
        dataset=True,
        mode='overwrite'
    )

    print(f"Data for {table_name} written to {s3_table_path}")
print('done')